In [ ]:

import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import re
import pyLDAvis.gensim
import  numpy  as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
import string
warnings.filterwarnings('ignore')

plt.style.use('ggplot')


# FUNCTIONS

# LOWER CASE DF

In [ ]:
import nltk


from nltk.tokenize import word_tokenize

def lower_case_df(text_to_lower):
    if isinstance(text_to_lower, str):
        tokens = word_tokenize(text_to_lower)
        lowercased_tokens = [token.lower() for token in tokens]
        return ' '.join(lowercased_tokens)
    else:
        return text_to_lower

# Example usage:
random_text = 'ThiS iS A tExT tO TEsT ThE fUnCTioN oF LOWerINg cAsE oF aLL LeTTErS'
lowercased_string = lower_case_df(random_text)

print(random_text)
print("--------------------------------------------------------------------")
print(lowercased_string)  


### REMOVE HTLM SIGNS

In [ ]:
def remove_htmlTags_punctuation(text):
  
    # Define the regular expression pattern for French punctuation marks
    punctuation_pattern = r'[,?!;:…()\[\]«»—/""\'\']'
    
    # Remove HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text_without_tags = soup.get_text()
    
    # Use re.sub() to remove the matched punctuation marks
    cleaned_text = re.sub(punctuation_pattern, '', text_without_tags)
    
    return cleaned_text

# Example usage:
html_text = '<p>Voici un exemple de texte avec des ponctuations et des balises HTLM : «Salut !» et \'Bonjour !\'</p>'
cleaned_text = remove_htmlTags_punctuation(html_text)
print(html_text)
print("--------------------------------------------------------------------------------------------------------")
print(cleaned_text)


### LEMMATIZER

In [ ]:
def lemmatizer(text):
    # Tokenization
    tokens = nltk.word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens 
              if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a single string
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

# Sample text
text = "This is an example sentence that we'll use for text cleaning. It includes stopwords and different verb tenses."


cleaned_text = lemmatizer(text)

print(text)
print("------------------------------------------------------------------------------------------------------------------------")

print(cleaned_text)

# ALL PREPROCESS FUNCTIONS INTO ONE

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re

def preprocessed_text(text):
    # Define the regular expression pattern for French punctuation marks
    punctuation_pattern = r'[,?!;:…()\[\]«»—/""\'\']'
    
    # Remove HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text_without_tags = soup.get_text()
    
    # Use re.sub() to remove the matched punctuation marks
    cleaned_text = re.sub(punctuation_pattern, '', text_without_tags)
    
    # Tokenization
    tokens = nltk.word_tokenize(cleaned_text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens 
              if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

# Sample text with HTML tags and punctuation
sample_text = """
<html>
<head><title>Sample Text</title></head>
<body>
<p>This is an example of text with HTML tags, punctuation, and some stop words. It's a sample text for preprocessing.</p>
</body>
</html>
"""

# Call the preprocessed_text function to preprocess the sample text
preprocessed_sample = preprocessed_text(sample_text)

print(sample_text)
print("------------------------------------------------------------------------------------------------------------------------")
# Display the preprocessed text
print(preprocessed_sample)


### Getting a MLB Dataframe with the top X most frequents tags with atleast a 1 in the rows

In [ ]:

def filter_by_tags(df, all_tags, X):
    # Step 4: Take the first X tags from the list
    first_tags = all_tags[:X]

    # Step 5: Filter the DataFrame based on the first X tags
    filtered_df = df[df[first_tags].any(axis=1)]

    # Step 6: Drop all columns beyond the first X tags
    filtered_df = filtered_df.drop(columns=all_tags[X:])
    filtered_df = pd.DataFrame(filtered_df)
    
    return filtered_df
